# 1 x 2 Probability for Given English Premier League Match

This is a reusable notebook for calculating the probability of home win (1), draw (x) or away win (2), shorthanded 1 x 2, for any given Premier League match given the away and home records of a given club and the average home and away performance of all Premier League clubs.

# TABLE OF CONTENTS
- [1. Retrieve home and away tables](#retrieve-home-and-away-tables)
- [2. Clean DataFranes](#clean-dataframes)


# Retrieve home and away tables
Use epl_table_scraper to retrieve home and away tables from UnderStat.

The scraper defaults to the EPL table, but this can retrieve other leagues with the following inputs if desired:

1. Bundesliga - "https://understat.com/league/Bundesliga"
2. La Liga - "https://understat.com/league/La_liga"
3. Serie A - "https://understat.com/league/Serie_A"
4. Ligue 1 - "https://understat.com/league/Ligue_1"

eg.
```python
scraper = WebScraper(url="https://understat.com/league/La_liga")
```


In [3]:
import sys
sys.path.append('../scraper')

from epl_table_scraper import WebScraper


# Retrieve 'Home' table
scraper = WebScraper()
scraper.load_page()
scraper.click_element("home-away2")
home_data_frame = scraper.scrape_table()

scraper.close()

print("\n======================== HOME_DF ==========================\n")
print(home_data_frame)

# Retrieve 'Away' table
scraper = WebScraper()
scraper.load_page()
scraper.click_element("home-away3")
away_data_frame = scraper.scrape_table()

scraper.close()

print("\n======================== AWAY_DF ==========================\n")
print(away_data_frame)

scraper is locating element... check
element located:  <selenium.webdriver.remote.webelement.WebElement (session="d4ec7619148e287c51ca2ea2b483052e", element="f.A0B577EA4EF07D62B29812A777425557.d.C65FBA423F6E6C1A6D904DDC0B83A1D7.e.15")>
element html:  <input id="home-away2" type="radio" name="home-away" value="h">
clicked the button
checking table update...
confirmed table updated successfully
found table
closing driver...
driver closed

======================== HOME_DF ==========================

     №                     Team  M  W  D  L   G  GA PTS          xG  \
0    1                Brentford  6  5  1  0  18  11  16  15.41-2.59   
1    2                Liverpool  6  5  0  1  11   3  15  13.04+2.04   
2    3          Manchester City  5  4  1  0  12   6  13  11.92-0.08   
3    4                Tottenham  6  4  0  2  16   6  12  12.59-3.41   
4    5                 Brighton  6  3  3  0  11   8  12  11.82+0.82   
5    6                  Arsenal  5  3  2  0  12   6  11  14.44+2.44   
6

# 2. Clean DataFranes

Now with that we have the data scraped and ready from Understat, we want to prepare to agregate some statistics for the whole league. We need to:
- (1) remove unneeded columns
- (2) calculate new columns for:
    - (a) goals per match scored
    - (b) goals per match conceded